# Loan Repayment 
## Exploratory Data Analysis (EDA)
<br>
<font size='3'>
Problem statement:<br>
Banks, fintech companies, and other institutions that provide loans rigourously need to analyze the credit score for all clients. There are mechanisms such as underwriting reports which assess the fraud risk for a given potential borrower. However, these risk scoring mechanisms only provide information whether to approve or not approve a certain loan. There are greater risk issues with funded loans where the borrower defaults (do not pay off). 
<br><br>
Hence, this loan repayment task is consequential towards objectively assessing the risk of whether a funded borrower can actually pay off their loans. In essence, this task ultimately results in creating <b>predictive models that can accurately predict whether a funded borrower will pay off (True) or default (False).</b> 
<br><br>
This notebook is the first part of loan repayment task which includes data transformations and exploring actionable data for interesting insights. Data transformation include cleaning unimportant records and columns, aggregating columns to create new parameter(s), and join data together with other datasets to come up with a set of parameters that will be useful for storytelling and later imported for the modelling section.
<br><br>
<b>Note: Please go through this notebook before reviewing modelling.ipynb</b>
</font>

In [1]:
import os
# Change directory to this file's directory
this_path = globals()['_dh'][-1]
os.chdir(this_path)
print("This file's directory:", os.getcwd())
# Change current path to parent of this file's directory
# to access all modules from parent
os.chdir('..')
source_path = os.getcwd()
print("Parent directory:", source_path)

This file's directory: /home/mattkhoo/Git-Loan-Repayment-EDA-Predictive/notebooks
Parent directory: /home/mattkhoo/Git-Loan-Repayment-EDA-Predictive


In [2]:
from main.data_loader import DataLoader
import main.viz_utils as viz
%matplotlib inline

In [3]:
# Load feather data
os.chdir(source_path)
feather_path = os.path.join(str(os.getcwd()),'data-feathers') 
for _, _, files in os.walk(feather_path):
    for file in files: print(file)

clarity_underwriting_variables
payment
loan


## Data Transformation on Loan 

<font size="3"> 
Reasoning: The chosen columns are deemed not useful for loan repayment analysis. Columns such as 'originated' and 'approved' precede (chronologically) the column 'isFunded'. Since this is a loan repayment problem, <b>we only care about funded loans</b>. Date columns are not relevant since there is no date data on when each loan is actually funded and when the loan is originally supposed to be paid. Hence, the punctuality of loan repayments cannot be measured. As for identification columns, only primary/foreign keys are useful for joins later, hence anon_ssn is removed. 
<br><br>
Further transformation will be done together with the other datasets after their respective transformations.
</font>

In [4]:
# Load loan data
loan_feather_path = os.path.join(feather_path, 'loan')
loan_manager = DataLoader()
loan_manager.load_feather(loan_feather_path)
# loan_manager.display(10, False)  # display last n rows

In [5]:
# Drop unused columns
dropped = ['anon_ssn', 'applicationDate', 'originated', 'originatedDate', 'approved']
loan_manager.drop_column(dropped)
loan_manager.display_types()

loanId                               object
payFrequency                         object
apr                                 float64
nPaidOff                            float64
isFunded                              int64
loanStatus                           object
loanAmount                          float64
originallyScheduledPaymentAmount    float64
state                                object
leadType                             object
leadCost                              int64
fpStatus                             object
clarityFraudId                       object
hasCF                                 int64
dtype: object

In [6]:
loan_manager.data.isFunded.unique()

array([0, 1])

In [7]:
loan_manager.single_eqfilter('isFunded', 1)  # get all funded loans
loan_manager.drop_column(['isFunded'])  # now we can drop isFunded
print("Shape of all funded loan data:", loan_manager.data.shape)

Shape of all funded loan data: (38982, 13)


## Data transformation for Payment

<font size='3'>There are only three important columns in payment, mainly for aggregation:</font>

- <font size='3'>loanId: </font>         
    - As a foreign key to join with loan data and basis for aggregation
    <br><br>
 - <font size='3'>paymentAmount:  </font> 
   - Aggregated sum for each loanID
   <br><br>
 - <font size='3'>paymentStatus:</font>    
   - Aggregate for only 'Checked' status


In [8]:
# Load payment data
payment_feather_path = os.path.join(feather_path, 'payment')
payment_manager = DataLoader()
payment_manager.load_feather(payment_feather_path)
# payment_manager.display(10, False) # display last n rows

<font size='4'>The intuition behind these transformations:</font>

 - <font size='3'> Enforces a focused scope on payments for loan repayment.</font><br><br>
 - <font size='3'><b>Only consider loan payments that are actually received by the company.</b></font><br><br>
 - <font size='3'>Aggregation of total successful payments yields a grouped sum for each loan.</font><br><br>
 - <font size='3'>Join with the loan dataset later will yield more meaningful results.</font>


In [9]:
# Filter 'Checked' payment status only
payment_manager.single_eqfilter('paymentStatus', 'Checked')
# Filter to only choose loanId and paymentAmount columns
chosen = ['loanId', 'paymentAmount']
payment_manager.chosen_cols(chosen)
print("Shape of all checked payments: ",payment_manager.data.shape)

Shape of all checked payments:  (209621, 2)


In [10]:
# Sum paymentAmount by loanId
group = ['loanId']
target = ['paymentAmount']
agg_payment = payment_manager.groupby_sum(group, target)    # returns a dataframe of summed loans
print("Shape of all loans with at least one successful payment:", agg_payment.shape)

Shape of all loans with at least one successful payment: (32168, 2)


In [11]:
agg_payment.head(10)

,loanId,paymentAmount
0,LL-I-00000021,1865.61
1,LL-I-00000231,258.64
2,LL-I-00000238,1162.60
3,LL-I-00000308,61.59
4,LL-I-00000419,226.31
5,LL-I-00001902,1272.00
6,LL-I-00002130,270.70
7,LL-I-00002148,200.74
8,LL-I-00002583,461.69
9,LL-I-00002680,46.03


In [12]:
"""
Now we need to join the loan data and aggregated payments to provide total payments of a loan along with other loan columns.
The intuition is to left join loan data with aggregated payments, as some funded loans might not have any payment at all.
Hence, resulting join will leave some records with blanks on paymentAmount.
"""
joined_lp = loan_manager.left_join(agg_payment, 'loanId')

'\nNow we need to join the loan data and aggregated payments to provide total payments of a loan along with other loan columns.\nThe intuition is to left join loan data with aggregated payments, as some funded loans might not have any payment at all.\nHence, resulting join will leave some records with blanks on paymentAmount.\n'

In [13]:
# Load underwriting data
undw_feather_path = os.path.join(feather_path, 'clarity_underwriting_variables')
undw_manager = DataLoader()
undw_manager.load_feather(undw_feather_path)
# undw_manager.display(10, False) # display last n rows